### **Setting GEE environnement**
Due to compatibility issues it is recommended to use a dedicated environment for the geemap library.

See instructions on how to do this here: https://geemap.org/installation/

In [1]:
# Install the libraries if they are not installed (you can remove these lines if you have already installed them)
# !pip install earthengine-api geemap

# Charging the libraries
import ee
import geemap

# Authenticate and initialize Earth Engine session
# ee.Authenticate()  # Only required the first time or when the access token expires
ee.Initialize()

### **Looping for export NDVI for each ROI**

In [6]:
# Define ROIs in a dictionary with a 10km buffer
rois = {
    'Cotriguacu': ee.FeatureCollection('users/caslumali/UE801_Stage/Cotriguacu_10km'),
    'Guaviare': ee.FeatureCollection('users/caslumali/UE801_Stage/Guaviare_10km'),
    'MDD': ee.FeatureCollection('users/caslumali/UE801_Stage/MDD_10km'),
    'Paragominas': ee.FeatureCollection('users/caslumali/UE801_Stage/Paragominas_10km'),
}

# Set the MapBiomas Amazônia collection
mapbiomas_amazonia = 'projects/mapbiomas-raisg/public/collection5/mapbiomas_raisg_panamazonia_collection5_integration_v1'
mapbiomas = ee.Image(mapbiomas_amazonia)

In [ ]:
# Loop for each ROI
for roi_name, roi in rois.items():
    print(f'Processing {roi_name}...')
    for year in range(1985, 2023):  # Adjust the range as needed
        bandName = f'classification_{year}'

        # Select and clip the specific band for the year
        image = mapbiomas.select([bandName]).clip(roi.geometry())
        
        description = f'Mapbiomas_{roi_name}_{year}'
        fileNamePrefix = f'{roi_name}_Mapbiomas_{year}'
        
        # Start the export task
        task = ee.batch.Export.image.toDrive(
            image=image,
            description=description,
            folder=f'Mapbiomas_Amazonia_{roi_name}',
            fileNamePrefix=fileNamePrefix,
            scale=30,
            region=roi.geometry(),
            crs='EPSG:4326',
            maxPixels=1e13
        )
        task.start()
        print(f'Export task started for {roi_name} in the year: {year}')